 # To view NYC Crime 2020 dashboard please go to: 

https://public.tableau.com/views/NYCCrime2020Dashboard/StoryofNYCCrimein2020?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

# NYC Crime in 2020 Dataset Cleanup

Four datasets were combined and cleaned to make a complete crime record for NYC Neighborhoods 2020.

NYPD Complaint Data Link (updated May 3, 2021): https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

NYPD Shooting Data Link (updated May 3, 2021): https://data.cityofnewyork.us/Public-Safety/NYPD-Shooting-Incident-Data-Historic-/833y-fsy8

MODZCTA Link (updated May 19, 2020): https://data.cityofnewyork.us/Health/Modified-Zip-Code-Tabulation-Areas-MODZCTA-/pri4-ifjk

Neighborhood Data Link: https://raw.githubusercontent.com/nychealth/coronavirus-data/master/latest/last7days-by-modzcta.csv

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree

import warnings
warnings.filterwarnings("ignore")


### Import NYPD Complaint Data

In [2]:
df_crime = pd.read_csv("NYPD_Complaint_Data_Historic.csv")

In [3]:
df_crime.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,700381962,05/28/2015,15:00:00,NaN,NaN,46.0,06/01/2015,578,HARRASSMENT 2,638.0,...,M,NaN,40.845868,-73.915888,"(40.84586773, -73.915888033)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F
1,642234217,10/28/2013,13:50:00,10/28/2013,13:50:00,120.0,10/28/2013,351,CRIMINAL MISCHIEF & RELATED OF,259.0,...,NaN,NaN,40.627061,-74.077149,"(40.627060894, -74.077149232)",PATROL BORO STATEN ISLAND,NaN,45-64,WHITE,M
2,242465164,05/09/2012,20:50:00,05/09/2012,21:00:00,24.0,05/09/2012,236,DANGEROUS WEAPONS,782.0,...,NaN,NaN,40.800966,-73.969047,"(40.800965968, -73.969047272)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E
3,927207428,01/03/2014,13:30:00,01/03/2014,13:35:00,108.0,01/03/2014,109,GRAND LARCENY,409.0,...,M,NaN,40.745242,-73.894253,"(40.745241809, -73.894253382)",PATROL BORO QUEENS NORTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,M
4,492142357,04/13/2016,00:00:00,NaN,NaN,40.0,04/13/2016,351,CRIMINAL MISCHIEF & RELATED OF,258.0,...,NaN,NaN,40.810352,-73.924942,"(40.810351863, -73.924942326)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E


In [4]:
df_crime.shape

(7396619, 35)

Before cleanup there are 7396619 rows and 35 columns.

### Missing Values

In [5]:
df_crime.isnull().sum()

CMPLNT_NUM                 0
CMPLNT_FR_DT             655
CMPLNT_FR_TM              48
CMPLNT_TO_DT         1707480
CMPLNT_TO_TM         1702808
ADDR_PCT_CD             2166
RPT_DT                     0
KY_CD                      0
OFNS_DESC              18831
PD_CD                   6328
PD_DESC                 6328
CRM_ATPT_CPTD_CD           7
LAW_CAT_CD                 0
BORO_NM                11379
LOC_OF_OCCUR_DESC    1546078
PREM_TYP_DESC          40904
JURIS_DESC                 0
JURISDICTION_CODE       6328
PARKS_NM             7368895
HADEVELOPT           7049021
HOUSING_PSA          6828743
X_COORD_CD             24064
Y_COORD_CD             24064
SUSP_AGE_GROUP       4802435
SUSP_RACE            3433894
SUSP_SEX             3567208
TRANSIT_DISTRICT     7232302
Latitude               24064
Longitude              24064
Lat_Lon                24064
PATROL_BORO             6785
STATION_NAME         7232302
VIC_AGE_GROUP        1638445
VIC_RACE                 309
VIC_SEX       

The columns that we want to remove nulls are as follows:
- BORO_NM: borough name
- CMPLNT_FR_DT: date of crime occurence
- Latitude/Longitude: the latitude and longitude of the crime 

In [6]:
#subsetting rows with no nulls in BORO_NM
df_crime = df_crime[pd.notnull(df_crime['BORO_NM'])]

In [7]:
#subsetting rows with no nulls in CMPLNT_FR_DT
df_crime = df_crime[pd.notnull(df_crime['CMPLNT_FR_DT'])]

In [8]:
#subsetting rows with no nulls in Lat_Lon
df_crime = df_crime[pd.notnull(df_crime['Lat_Lon'])]

In [9]:
#checking nulls again
df_crime.isnull().sum()

CMPLNT_NUM                 0
CMPLNT_FR_DT               0
CMPLNT_FR_TM              47
CMPLNT_TO_DT         1692459
CMPLNT_TO_TM         1687839
ADDR_PCT_CD                2
RPT_DT                     0
KY_CD                      0
OFNS_DESC              18684
PD_CD                      0
PD_DESC                    0
CRM_ATPT_CPTD_CD           7
LAW_CAT_CD                 0
BORO_NM                    0
LOC_OF_OCCUR_DESC    1540211
PREM_TYP_DESC          33949
JURIS_DESC                 0
JURISDICTION_CODE          0
PARKS_NM             7333302
HADEVELOPT           7015031
HOUSING_PSA          6794904
X_COORD_CD                 0
Y_COORD_CD                 0
SUSP_AGE_GROUP       4778903
SUSP_RACE            3415835
SUSP_SEX             3548076
TRANSIT_DISTRICT     7197485
Latitude                   0
Longitude                  0
Lat_Lon                    0
PATROL_BORO               78
STATION_NAME         7197485
VIC_AGE_GROUP        1631632
VIC_RACE                 306
VIC_SEX       

There are now no null values for the columns that matter to us. 

### Filter Date Range

The dataset is supposed only include dates from 2006 until the present. Furthermore, in order to work with a smaller dataset that is easier to process, we will choose the date range 2018 to 2020. 

First we need to convert CMPLNT_FR_DT to datetime format. 

In [10]:
#convert CMPLNT_FR_DT column to datetime, 'coerce' forces dates that are out-of-bounds to be null values
#out-of-bounds means dates are not within 584 years 

df_crime['CMPLNT_FR_DT'] = pd.to_datetime(df_crime['CMPLNT_FR_DT'], errors = 'coerce')

In [11]:
#filter for dates that are 2020-01-01 until the present 
df_crime = df_crime[df_crime['CMPLNT_FR_DT'] >= '2020-01-01']

In [12]:
#check the minimum date to make sure filtering worked
df_crime['CMPLNT_FR_DT'].min()

Timestamp('2020-01-01 00:00:00')

Now, df_crime has the correct date range (2020 - present). 

### Drop Duplicates

In [13]:
#how many rows there are
df_crime.shape

(404429, 35)

In [14]:
#drop duplicates
df_crime.drop_duplicates(subset = ['CMPLNT_NUM'], inplace = True)

In [15]:
#how many row there are without duplicates
df_crime.shape

(404429, 35)

The total number of instances of each identifier is the same as the number of rows in the dataframe so there are no duplicates.

### Import NYPD Shooting Data

In [16]:
df_shooting = pd.read_csv("NYPD_Shooting_Incident_Data__Historic_.csv")

In [17]:
df_shooting.head()

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,201575314,08/23/2019,22:10:00,QUEENS,103,0.0,NaN,False,NaN,NaN,NaN,25-44,M,BLACK,1037451,193561,40.697805,-73.808141,POINT (-73.80814071699996 40.697805308000056)
1,205748546,11/27/2019,15:54:00,BRONX,40,0.0,NaN,False,<18,M,BLACK,25-44,F,BLACK,1006789,237559,40.818700,-73.918571,POINT (-73.91857061799993 40.81869973000005)
2,193118596,02/02/2019,19:40:00,MANHATTAN,23,0.0,NaN,False,18-24,M,WHITE HISPANIC,18-24,M,BLACK HISPANIC,999347,227795,40.791916,-73.945480,POINT (-73.94547965999999 40.791916091000076)
3,204192600,10/24/2019,00:52:00,STATEN ISLAND,121,0.0,PVT HOUSE,True,25-44,M,BLACK,25-44,F,BLACK,938149,171781,40.638064,-74.166108,POINT (-74.16610830199996 40.63806398200006)
4,201483468,08/22/2019,18:03:00,BRONX,46,0.0,NaN,False,25-44,M,BLACK HISPANIC,18-24,M,BLACK,1008224,250621,40.854547,-73.913339,POINT (-73.91333944399999 40.85454734900003)


In [18]:
df_shooting.shape

(23568, 19)

Before cleanup there are 23,569 rows and 19 columns.

### Missing Data

In [19]:
df_shooting.isnull().sum()

INCIDENT_KEY                   0
OCCUR_DATE                     0
OCCUR_TIME                     0
BORO                           0
PRECINCT                       0
JURISDICTION_CODE              2
LOCATION_DESC              13581
STATISTICAL_MURDER_FLAG        0
PERP_AGE_GROUP              8459
PERP_SEX                    8425
PERP_RACE                   8425
VIC_AGE_GROUP                  0
VIC_SEX                        0
VIC_RACE                       0
X_COORD_CD                     0
Y_COORD_CD                     0
Latitude                       0
Longitude                      0
Lon_Lat                        0
dtype: int64

There is no missing information in the important columns for our analysis. 

### Filter Date Range

We will filter for the year 2020. 

In [20]:
#convert OCCUR_DATE column to datetime, 'coerce' forces dates that are out-of-bounds to be null values
#out-of-bounds means dates are not within 584 years 

df_shooting['OCCUR_DATE'] = pd.to_datetime(df_shooting['OCCUR_DATE'], errors = 'coerce')

In [21]:
#filter for dates that are 2020-01-01 until the present 
df_shooting = df_shooting[df_shooting['OCCUR_DATE'] >= '2020-01-01']

In [22]:
#check the minimum date to make sure filtering worked
df_shooting['OCCUR_DATE'].min()

Timestamp('2020-01-01 00:00:00')

### Drop Duplicates

Duplicates in this case mean multiple victims for one incident, so duplicates will be okay to keep. 

### Make columns to match NYPD Complaint Data

In [23]:
#first check data types of each attribute
df_shooting.dtypes

INCIDENT_KEY                        int64
OCCUR_DATE                 datetime64[ns]
OCCUR_TIME                         object
BORO                               object
PRECINCT                            int64
JURISDICTION_CODE                 float64
LOCATION_DESC                      object
STATISTICAL_MURDER_FLAG              bool
PERP_AGE_GROUP                     object
PERP_SEX                           object
PERP_RACE                          object
VIC_AGE_GROUP                      object
VIC_SEX                            object
VIC_RACE                           object
X_COORD_CD                         object
Y_COORD_CD                         object
Latitude                          float64
Longitude                         float64
Lon_Lat                            object
dtype: object

In [24]:
#rename certain columns to match NYPD complaint data 
df_shooting.rename(columns = {'OCCUR_DATE': 'CMPLNT_FR_DT', 'INCIDENT_KEY':'CMPLNT_NUM', 'STATISTICAL_MURDER_FLAG':'OFNS_DESC'}, inplace = True)

In [25]:
#convert lat and Lon to strings
df_shooting['Latitude'] = df_shooting['Latitude'].astype(str)
df_shooting['Longitude'] = df_shooting['Longitude'].astype(str)

In [26]:
#need to make "Lat_Lon" column
df_shooting['Lat_Lon'] = df_shooting['Latitude'].str.cat(df_shooting['Longitude'], sep = ",")

In [27]:
#insert "()" in the "lat_Lon" column
df_shooting['Lat_Lon'] = ('(' + df_shooting['Lat_Lon'] + ')')

In [28]:
#Label OFNS_DESC column as being either SHOOTING INCIDENT or SHOOTING INCIDENT MURDER
#If it is false it is a SHOOTING INCIDENT, if it is true it is a SHOOTING INCIDENT MURDER

df_shooting['OFNS_DESC'] = df_shooting['OFNS_DESC'].apply(lambda x: 'SHOOTING INCIDENT' if x == False else 'SHOOTING INCIDENT MURDER')

In [29]:
df_shooting.head()

,CMPLNT_NUM,CMPLNT_FR_DT,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,OFNS_DESC,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat,Lat_Lon
21626,214366724,2020-06-19,01:36:00,QUEENS,103,0.0,COMMERCIAL BLDG,SHOOTING INCIDENT,NaN,NaN,NaN,25-44,M,BLACK,1016807,179007,40.657957922000044,-73.88266012399998,POINT (-73.88266012399998 40.657957922000044),"(40.657957922000044,-73.88266012399998)"
21627,215350310,2020-07-15,17:54:00,BROOKLYN,73,0.0,MULTI DWELL - APT BUILD,SHOOTING INCIDENT,NaN,NaN,NaN,<18,M,BLACK,1011811,246833,40.844139945000045,-73.90038861799998,POINT (-73.90038861799998 40.84413994500005),"(40.844139945000045,-73.90038861799998)"
21628,211362502,2020-03-20,20:19:00,MANHATTAN,25,0.0,NaN,SHOOTING INCIDENT,NaN,NaN,NaN,25-44,M,BLACK,1009320,244794,40.83855090900005,-73.90939937399997,POINT (-73.90939937399997 40.83855090900005),"(40.83855090900005,-73.90939937399997)"
21629,221490639,2020-12-05,23:35:00,QUEENS,110,0.0,NaN,SHOOTING INCIDENT,NaN,NaN,NaN,65+,M,ASIAN / PACIFIC ISLANDER,1001350,201326,40.71926178300004,-73.93831327799995,POINT (-73.93831327799995 40.71926178300004),"(40.71926178300004,-73.93831327799995)"
21630,213050797,2020-05-13,20:24:00,BROOKLYN,67,0.0,NaN,SHOOTING INCIDENT MURDER,NaN,NaN,NaN,25-44,M,BLACK,1051960,160392,40.606664855000076,-73.75614960299998,POINT (-73.75614960299998 40.606664855000076),"(40.606664855000076,-73.75614960299998)"


In [30]:
#select columns
df_shooting = df_shooting[['CMPLNT_NUM', 'CMPLNT_FR_DT','OFNS_DESC','Lat_Lon']]

In [31]:
#append df_shooting rows to df_crime
df_crime = df_shooting.append(df_crime)

In [32]:
df_crime

,CMPLNT_NUM,CMPLNT_FR_DT,OFNS_DESC,Lat_Lon,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,...,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
21626,214366724,2020-06-19,SHOOTING INCIDENT,"(40.657957922000044,-73.88266012399998)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21627,215350310,2020-07-15,SHOOTING INCIDENT,"(40.844139945000045,-73.90038861799998)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21628,211362502,2020-03-20,SHOOTING INCIDENT,"(40.83855090900005,-73.90939937399997)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21629,221490639,2020-12-05,SHOOTING INCIDENT,"(40.71926178300004,-73.93831327799995)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21630,213050797,2020-05-13,SHOOTING INCIDENT MURDER,"(40.606664855000076,-73.75614960299998)",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7396612,116305536,2020-01-02,GRAND LARCENY,"(40.81593767200008, -73.95796004899995)",17:00:00,01/02/2020,17:05:00,26.0,01/02/2020,109.0,...,NaN,NaN,NaN,40.815938,-73.957960,PATROL BORO MAN NORTH,NaN,18-24,BLACK HISPANIC,M
7396613,234726969,2020-01-05,CRIMINAL MISCHIEF & RELATED OF,"(40.80404600700007, -73.93662010299995)",05:00:00,01/05/2020,07:50:00,25.0,01/05/2020,351.0,...,UNKNOWN,U,NaN,40.804046,-73.936620,PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,D
7396616,950546587,2020-01-04,CRIMINAL TRESPASS,"(40.81242379100007, -73.94953058799997)",20:30:00,01/04/2020,20:34:00,32.0,01/04/2020,352.0,...,BLACK,M,NaN,40.812424,-73.949531,PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,E
7396617,531630504,2020-01-03,FELONY ASSAULT,"(40.71521116400004, -73.74887699299995)",08:30:00,01/03/2020,08:45:00,105.0,01/03/2020,106.0,...,UNKNOWN,U,NaN,40.715211,-73.748877,PATROL BORO QUEENS SOUTH,NaN,18-24,ASIAN / PACIFIC ISLANDER,M


### Make a Zipcode & Neighborhood Column

It will be easier to see regional data by zipcode. However, there are only latitude and longitude coordinates. 

Using a KD Tree model we can closely approximate these zipcodes. 

In [33]:
#Reset index 
df_crime.reset_index(inplace=True)

In [34]:
#make tuples of the Lat_Lon column
df_crime['Lat_Lon'] = df_crime['Lat_Lon'].apply(lambda x: eval(str(x)))

In [35]:
#Create location dataframe from census data
#code reference: https://www.codementor.io/@bobhaffner/reverse-geocoding-bljjp5byw
#zip reference: https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2020_Gazetteer/2020_Gaz_zcta_national.zip

#create location dataframe from census data 
df_locations = pd.read_csv('2020_Gaz_zcta_national.zip', dtype={'GEOID':'str'}, sep='\t')

#column cleanup
df_locations.columns = df_locations.columns.str.strip()

#print df_locations to inspect it
print (len(df_locations)) 
print(df_locations.head())

33144
   GEOID      ALAND   AWATER  ALAND_SQMI  AWATER_SQMI   INTPTLAT  INTPTLONG
0  00601  166659744   799292      64.348        0.309  18.180555 -66.749961
1  00602   79307538  4428428      30.621        1.710  18.361945 -67.175597
2  00603   81887203   181412      31.617        0.070  18.455183 -67.119887
3  00606  109579950    12487      42.309        0.005  18.158327 -66.932928
4  00610   93013430  4172059      35.913        1.611  18.294032 -67.127156


In [36]:
#KDT Model
kdt = KDTree(df_locations[['INTPTLAT', 'INTPTLONG']])

In [37]:
#Create loop to make a list of zip codes that correspond to the lat and long in 'lat_Lon' column
list1=[]
for i in range(len(df_crime['Lat_Lon'])):
    nyc_point = np.array((df_crime['Lat_Lon'][i]))
    nyc_point_kdt = np.expand_dims(nyc_point, axis=0)
    list1.append(df_locations.loc[kdt.query(nyc_point_kdt, k=1, return_distance=False)[0], 'GEOID'].values)

In [38]:
#need to "remove axes of length one in numpy array that was created"
#https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html
list1 = np.squeeze(list1)

In [39]:
#Make a new zipcode column
df_crime['zipcode'] = list1

In [40]:
df_crime.head()

,index,CMPLNT_NUM,CMPLNT_FR_DT,OFNS_DESC,Lat_Lon,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,zipcode
0,21626,214366724,2020-06-19,SHOOTING INCIDENT,"(40.657957922000044, -73.88266012399998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11239
1,21627,215350310,2020-07-15,SHOOTING INCIDENT,"(40.844139945000045, -73.90038861799998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10457
2,21628,211362502,2020-03-20,SHOOTING INCIDENT,"(40.83855090900005, -73.90939937399997)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10456
3,21629,221490639,2020-12-05,SHOOTING INCIDENT,"(40.71926178300004, -73.93831327799995)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11222
4,21630,213050797,2020-05-13,SHOOTING INCIDENT MURDER,"(40.606664855000076, -73.75614960299998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11691


### Modified Zip Code Tabulation Areas (MODZCTA) - Match Zipcode Data with MODZCTA

In regional data it is recommended to represent zipcode data as MODZCTA data, which are basically modified zipcodes that represent neighborhoods. We will download this data from NYC Open Data. 

In [41]:
#Data Source
#https://data.cityofnewyork.us/Health/Modified-Zip-Code-Tabulation-Areas-MODZCTA-/pri4-ifjk/data?no_mobile=true

In [42]:
#read in data to make MODZCTA dataframe
df_modzcta = pd.read_csv('Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv')

In [43]:
pd.set_option('display.max_rows', 179) #allow us to see all rows
df_modzcta

,MODZCTA,label,ZCTA,pop_est,the_geom
0,10001,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...
1,10002,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...
2,10003,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...
3,10026,10026,10026,39363,MULTIPOLYGON (((-73.96200720826798 40.80550908...
4,10004,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...
5,10005,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...
6,10006,10006,10006,3454,MULTIPOLYGON (((-74.01250844578733 40.70676972...
7,10007,10007,"10007, 10278, 10279",7023,MULTIPOLYGON (((-74.0099841276905 40.709805680...
8,10009,10009,10009,57925,MULTIPOLYGON (((-73.98863576728334 40.72293099...
9,10010,10010,10010,33730,MULTIPOLYGON (((-73.9797860547436 40.734962793...


The 'label' column corresponds to zipcodes. 

In [44]:
#remove null values 
df_modzcta.dropna(inplace= True)

In [45]:
df_modzcta

,MODZCTA,label,ZCTA,pop_est,the_geom
0,10001,"10001, 10118","10001, 10119, 10199",23072,MULTIPOLYGON (((-73.98774438827894 40.74406551...
1,10002,10002,10002,74993,MULTIPOLYGON (((-73.99750380833868 40.71407144...
2,10003,10003,10003,54682,MULTIPOLYGON (((-73.98863576728334 40.72293099...
3,10026,10026,10026,39363,MULTIPOLYGON (((-73.96200720826798 40.80550908...
4,10004,10004,10004,3028,MULTIPOLYGON (((-74.00826694529984 40.70771979...
5,10005,10005,"10005, 10271",8831,MULTIPOLYGON (((-74.00782636323468 40.70308666...
6,10006,10006,10006,3454,MULTIPOLYGON (((-74.01250844578733 40.70676972...
7,10007,10007,"10007, 10278, 10279",7023,MULTIPOLYGON (((-74.0099841276905 40.709805680...
8,10009,10009,10009,57925,MULTIPOLYGON (((-73.98863576728334 40.72293099...
9,10010,10010,10010,33730,MULTIPOLYGON (((-73.9797860547436 40.734962793...


In [46]:
#select only the MODZCTA, label and pop_est columns
df_modzcta = df_modzcta[['MODZCTA', 'label', 'pop_est']]

In [47]:
df_modzcta

,MODZCTA,label,pop_est
0,10001,"10001, 10118",23072
1,10002,10002,74993
2,10003,10003,54682
3,10026,10026,39363
4,10004,10004,3028
5,10005,10005,8831
6,10006,10006,3454
7,10007,10007,7023
8,10009,10009,57925
9,10010,10010,33730


In [48]:
#Need to convert fields with multiple data into multiple rows
#Code reference: https://sureshssarda.medium.com/pandas-splitting-exploding-a-column-into-multiple-rows-b1b1d59ea12e
new_df = pd.DataFrame(df_modzcta['label'].str.split(',').tolist(), index=df_modzcta['MODZCTA']).stack()
new_df = new_df.reset_index([0, 'MODZCTA'])
new_df.columns = ['MODZCTA', 'zipcode']
new_df

,MODZCTA,zipcode
0,10001,10001
1,10001,10118
2,10002,10002
3,10003,10003
4,10026,10026
...,...,...
178,11433,11433
179,11434,11434
180,11435,11435
181,11694,11694


In [49]:
#merge new_df and df_modzcta using a left join
df_modzcta = new_df.merge(df_modzcta, how = 'left', on ='MODZCTA')

In [50]:
#select columns
df_modzcta = df_modzcta[['MODZCTA', 'zipcode','pop_est']]

In [51]:
df_modzcta.head()

,MODZCTA,zipcode,pop_est
0,10001,10001,23072
1,10001,10118,23072
2,10002,10002,74993
3,10003,10003,54682
4,10026,10026,39363


### Neighborhood Names - Match Neighborhood Names to MODZCTA

It is also useful to match neighborhood names to MODZCTA data. So we will use the NYC Cvodi-19 dataset which contains this. 

In [52]:
#Data Source
#https://raw.githubusercontent.com/nychealth/coronavirus-data/master/latest/last7days-by-modzcta.csv

In [53]:
#import data to make Neighborhood Name Dataframe
df_names = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/latest/last7days-by-modzcta.csv')

In [54]:
df_names.head()

,modzcta,modzcta_name,label,lat,lon,percentpositivity_7day,people_tested,people_positive,median_daily_test_rate,adequately_tested,daterange
0,10001,Chelsea/NoMad/West Chelsea,"10001, 10118",40.750693,-73.997137,0.87,1036,9,738.8,Yes,April 29-May 5
1,10002,Chinatown/Lower East Side,10002,40.715781,-73.986176,1.66,2343,39,543.0,Yes,April 29-May 5
2,10003,East Village/Gramercy/Greenwich Village,10003,40.731825,-73.989164,0.96,3220,31,1106.0,Yes,April 29-May 5
3,10004,Financial District,10004,40.703675,-74.013106,0.94,106,1,403.8,Yes,April 29-May 5
4,10005,Financial District,10005,40.706092,-74.008861,0.72,276,2,502.4,Yes,April 29-May 5


In [55]:
#select columns 
df_names = df_names[['modzcta','modzcta_name','label']]

In [56]:
df_names.head()

,modzcta,modzcta_name,label
0,10001,Chelsea/NoMad/West Chelsea,"10001, 10118"
1,10002,Chinatown/Lower East Side,10002
2,10003,East Village/Gramercy/Greenwich Village,10003
3,10004,Financial District,10004
4,10005,Financial District,10005


In [57]:
#rename modzcta column
df_names.columns = ['MODZCTA','NeighborhoodName','ZipsThatMakeUpMODZCTA']

In [58]:
#merge df_names and df_modzcta into new dataframe called df_nyclocation
df_nyclocation = df_modzcta.merge(df_names, how = 'left', on ='MODZCTA')

In [59]:
df_nyclocation.head()

,MODZCTA,zipcode,pop_est,NeighborhoodName,ZipsThatMakeUpMODZCTA
0,10001,10001,23072,Chelsea/NoMad/West Chelsea,"10001, 10118"
1,10001,10118,23072,Chelsea/NoMad/West Chelsea,"10001, 10118"
2,10002,10002,74993,Chinatown/Lower East Side,10002
3,10003,10003,54682,East Village/Gramercy/Greenwich Village,10003
4,10026,10026,39363,Central Harlem (South),10026


In [60]:
#check column types, for MODZCTA we want it to be a string not an integer
df_nyclocation.dtypes

MODZCTA                   int64
zipcode                  object
pop_est                   int64
NeighborhoodName         object
ZipsThatMakeUpMODZCTA    object
dtype: object

In [61]:
#convert MODZCTA column into strings
df_nyclocation['MODZCTA'] = df_nyclocation['MODZCTA'].astype(str)

### Merge NYC Location Data to Crime Data

In [62]:
#merge df_nyclocation with df_crime
df_crime = df_crime.merge(df_nyclocation, how = 'left', on = 'zipcode')

In [63]:
#search for missing values in 'NeighborhoodName': nulls in this column means that these locations did not occur in a specific neighborhood but maybe in a park or airport.
#search for missing values in 'OFNS_DESC': we will sort by crime description so null values are not very specific and should be removed. 
df_crime.isnull().sum()

index                         0
CMPLNT_NUM                    0
CMPLNT_FR_DT                  0
OFNS_DESC                     5
Lat_Lon                       0
CMPLNT_FR_TM               1942
CMPLNT_TO_DT              39478
CMPLNT_TO_TM              39359
ADDR_PCT_CD                1942
RPT_DT                     1942
KY_CD                      1942
PD_CD                      1942
PD_DESC                    1942
CRM_ATPT_CPTD_CD           1942
LAW_CAT_CD                 1942
BORO_NM                    1942
LOC_OF_OCCUR_DESC         67202
PREM_TYP_DESC              2605
JURIS_DESC                 1942
JURISDICTION_CODE          1942
PARKS_NM                 403718
HADEVELOPT               404834
HOUSING_PSA              375954
X_COORD_CD                 1942
Y_COORD_CD                 1942
SUSP_AGE_GROUP            93841
SUSP_RACE                 93841
SUSP_SEX                  93841
TRANSIT_DISTRICT         399429
Latitude                   1942
Longitude                  1942
PATROL_B

In [64]:
#remove nulls from 'NeighborhoodName' column
df_crime = df_crime[pd.notnull(df_crime['NeighborhoodName'])]

In [65]:
#remove nulls from 'OFNS_DESC ' column
df_crime = df_crime[pd.notnull(df_crime['OFNS_DESC'])]

In [66]:
df_crime.isnull().sum()

index                         0
CMPLNT_NUM                    0
CMPLNT_FR_DT                  0
OFNS_DESC                     0
Lat_Lon                       0
CMPLNT_FR_TM               1900
CMPLNT_TO_DT              37373
CMPLNT_TO_TM              37259
ADDR_PCT_CD                1900
RPT_DT                     1900
KY_CD                      1900
PD_CD                      1900
PD_DESC                    1900
CRM_ATPT_CPTD_CD           1900
LAW_CAT_CD                 1900
BORO_NM                    1900
LOC_OF_OCCUR_DESC         64003
PREM_TYP_DESC              2467
JURIS_DESC                 1900
JURISDICTION_CODE          1900
PARKS_NM                 386548
HADEVELOPT               387654
HOUSING_PSA              359113
X_COORD_CD                 1900
Y_COORD_CD                 1900
SUSP_AGE_GROUP            90020
SUSP_RACE                 90020
SUSP_SEX                  90020
TRANSIT_DISTRICT         383148
Latitude                   1900
Longitude                  1900
PATROL_B

In [67]:
df_crime.head()

,index,CMPLNT_NUM,CMPLNT_FR_DT,OFNS_DESC,Lat_Lon,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,...,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,zipcode,MODZCTA,pop_est,NeighborhoodName,ZipsThatMakeUpMODZCTA
0,21626,214366724,2020-06-19,SHOOTING INCIDENT,"(40.657957922000044, -73.88266012399998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11239,11239,13244.0,East New York,11239
1,21627,215350310,2020-07-15,SHOOTING INCIDENT,"(40.844139945000045, -73.90038861799998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10457,10457,74554.0,Belmont/Claremont/Mount Hope/Tremont,10457
2,21628,211362502,2020-03-20,SHOOTING INCIDENT,"(40.83855090900005, -73.90939937399997)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10456,10456,94218.0,Claremont/Morrisania,10456
3,21629,221490639,2020-12-05,SHOOTING INCIDENT,"(40.71926178300004, -73.93831327799995)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11222,11222,36492.0,Greenpoint,11222
4,21630,213050797,2020-05-13,SHOOTING INCIDENT MURDER,"(40.606664855000076, -73.75614960299998)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11691,11691,67094.0,Edgemere/Far Rockaway,11691


All crimes in df_crime were committed in New York City

### Clean Dataset Description: NYPD_Complaint_Data_Historic_Clean.csv

- The dataset was filtered to include non-missing 2020 NYPD crime data (including shootings & murders) in NYC only. 
- There was a focus on 2020 due to the large size of the original data and it being the most recent. 
- Location and neighborhood data were added for each crime:
    - zipcode	
    - MODZCTA	
    - pop_est	
    - NeighborhoodName	
    - ZipsThatMakeUpMODZCTA

In [68]:
df_crime.shape

(389074, 41)

### Export Clean Dataset

In [69]:
df_crime.to_csv("NYPD_Complaint_Data_Historic_Clean.csv")